In [1]:
import pandas as pd
import numpy as np
import os 
import pickle
#from fuzzywuzzy import process, fuzz
from rapidfuzz import process, utils, fuzz
from collections import Counter
from tqdm import tqdm
pd.options.display.max_columns = None

# checking the dataset's data
Checking the data from the [given](https://www.kaggle.com/datasets/andrewmvd/spotify-playlists?select=spotify_dataset.csv) database.

In [2]:
data = pd.read_csv('../data/charts.csv', sep = ',', on_bad_lines='skip')
print(data.shape)
data.head()

(26173514, 9)


,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


In [6]:
data[['region', 'chart']].value_counts()

region         chart  
Brazil         top200     364516
Argentina      top200     364385
United States  top200     364184
Austria        top200     363925
Australia      top200     362189
                           ...  
India          viral50     51688
Ukraine        viral50     26778
Russia         viral50     26069
South Korea    viral50     16700
Luxembourg     top200       8962
Length: 139, dtype: int64

In [7]:
data[data['region'] == 'Spain'][['region', 'chart']].value_counts()

region  chart  
Spain   top200     358355
        viral50     90904
dtype: int64

In [12]:
data_new = data[(data['region'] == 'Spain')]

check if data has duplicated values

In [ ]:
data.columns

In [ ]:
data[['artist', 'song']].duplicated().sum()

In [ ]:
data = data.drop_duplicates(subset=['artist', 'song'])
print(data.shape)
data.head()

remove unnecesary columns

In [ ]:
data_new = data.drop(['link'], axis = 1)
print(data_new.shape)
data_new.head()

# check if the data is clean
It is possible that some artist are named similarly and create duplicated values

In [ ]:
data_new.dropna(subset=['artist'], inplace=True)
data_new.dropna(subset=['song'], inplace=True)
print(data_new.shape)
data_new.head()

In [ ]:
print('unique artists:',len(data_new['artist'].unique().tolist()),'; unique songs (with unique names):',len(data_new['song'].unique().tolist()))

making all `artist` lowercase, removing all spaces and weird characters

In [ ]:
data_new['artist_clean'] = data_new['artist'].str.strip().str.lower().str.replace('&', 'and').str.replace('  ', ' ').str.replace('-', '/').str.replace(' / ', '/')
print(len(data_new['artist_clean'].unique().tolist()), len(data_new['artist'].unique().tolist()))

doing the same with the `tracks`

In [ ]:
data_new['track_clean'] = data_new['song'].str.strip().str.lower().str.replace('&', 'and').str.replace('  ', ' ').str.replace(' / ', '/')
print(len(data_new['track_clean'].unique().tolist()), len(data_new['song'].unique().tolist()))

In [ ]:
data_new

In [ ]:
data_new = data_new.drop_duplicates(subset=['artist_clean', 'track_clean'])
print(data_new.shape)
data_new.head()

In [ ]:
def parecido_artistas(col, dict_artistas):   
    maximo = 0
    for key in dict_artistas.keys():
    #print(key)
        #print(genero)
        comparo = fuzz.ratio(key, col)
        if comparo > maximo:
            maximo = comparo
            resultado = key
    if maximo > 90:
        return resultado
    else:
        return np.nan

Fixing the `artist`s, in order to remove duplicated artist or typos -- replacing by the most common occurance by similarity.

In [ ]:
data_new[data_new['artist_clean'].str.contains('costello') == True]['artist_clean'].unique().tolist()[:10]

In [ ]:
fuzz.ratio('elvis costello and the attractions', 'elvis costello/the attractions')

In [ ]:
cuenta_artistas = Counter(artistas for artistas in data_new['artist_clean'])
print(len(cuenta_artistas))
cuenta_artistas

In [ ]:
dict_artistas = dict(cuenta_artistas)
dict_artistas = {k:v for k,v in dict_artistas.items() if v > 0}
print(len(dict_artistas))
dict_artistas#.get(3)

In [ ]:
data_test = data_new.sample(1000)

In [ ]:
data_test['artist'] = data_test.apply(lambda x: parecido_artistas(x['artist_clean'], dict_artistas), axis=1)
data_test[['artist_clean','artist']].value_counts()

In [ ]:
#data_test.to_csv('test.csv', index=False, sep=';')

In [ ]:
tqdm.pandas()

data_new['artist_clean_new'] = data_new.progress_apply(lambda x: parecido_artistas(x['artist_clean'], dict_artistas), axis=1)
data_new[['artist_clean','artist_clean_new']].value_counts()

In [ ]:
data_new['check'] = np.where(data_new['artist_clean_new'] == data_new['artist_clean'], 'same', 'different' )
data_new['check'].value_counts()

In [ ]:
data_new[data_new['check']== 'same']

Doing the same process with the `tracks`, in order to remove typos.

In [ ]:
data_new.head()

In [ ]:
data_new[data_new['track_clean'].str.contains("satisfaction") == True]['track_clean'].unique().tolist()[:10]

In [ ]:
cuenta_tracks = Counter(track for track in data_new['track_clean'])
print(len(cuenta_tracks))
cuenta_tracks

In [ ]:
dict_tracks = dict(cuenta_tracks)
dict_tracks = {k:v for k,v in dict_tracks.items() if v > 4}
print(len(dict_tracks))
dict_tracks

In [ ]:
def parecido_tracks(col, dict_tracks):   
    maximo = 0
    for key in dict_tracks.keys():
    #print(key)
        #print(genero)
        comparo = fuzz.ratio(key, col)
        if comparo > maximo:
            maximo = comparo
            resultado = key
    if maximo > 70:
        return resultado
    else:
        return np.nan

In [ ]:
data_new.head()

In [ ]:
tqdm.pandas()

data_new['track_clean_new'] = data_new.progress_apply(lambda x: parecido_tracks(x['track_clean'], dict_tracks), axis=1)
data_new[['track_clean','track_clean_new']].value_counts()

In [ ]:
data_new['check'] = np.where(data_new['track_clean_new'] == data_new['track_clean'], 'same', 'different')
data_new['check'].value_counts()

In [ ]:
data_new[data_new['check'] == 'different']

In [ ]:
print(len(data_new[' "trackname"'].unique().tolist()), len(data_new['track'].unique().tolist()))

# Conversion
I will save the input file as .pkl as the original one is too big for github.

In [ ]:
data_final = data_new.drop_duplicates(subset=['artist_clean', 'track_clean'])
print(data_final.shape)
data_final.head()

In [13]:
with open('../data/dataset_viral150.pickle', 'wb') as data_nowplaying:
    pickle.dump(data_new, data_nowplaying)